---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    #Numbers only
    a=df.str.extractall(r'((\d{1,2}[/-])?(\d{1,2}[/-])(\d{2,4}))')
    a=a.reset_index()

    #Named Months
    b=df.str.extractall(r'((\d{1,2}[a-z]*[?:, -.]* )?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -.]* )(\d{1,2}[a-z]*[?:, -.]* )?(\d{2,4}))')
    b=b.reset_index()

    #Year only
    c=df.str.extractall(r'((19|20)\d{2})')
    c=c.reset_index()
    cunique=c[~c['level_0'].isin(a['level_0'])] 
    cunique=cunique[~cunique['level_0'].isin(b['level_0'])]    

    #Convert dates with year only to 01/01/yyyy
    cunique=cunique.rename(columns={0:'YYYY'}).drop(labels=1, axis=1)
    cunique['MM']='01'
    cunique['DD']='01'
    cunique['Date']=pd.to_datetime(cunique['MM']+'/'+cunique['DD']+'/'+cunique['YYYY']) 

    #Convert named months to 01/01/yyyy
    convertmonth={'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',\
                  'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    columns=['DD_bef', 'MM', 'DD_aft', 'YYYY']
    b=b.rename(columns={1:'DD_bef', 2:'MM', 3:'DD_aft', 4:'YYYY'})
    b['DD_aft']=b['DD_aft'].astype(str)
    b['DD_aft']=b['DD_aft'].str.lstrip().str[:2]

    b['DD_bef']=b['DD_bef'].astype(str)
    b['DD_bef']=b['DD_bef'].str.lstrip().str[:2]
    b['MM']=b['MM'].str.lstrip().str[:3].replace(convertmonth)

    for i in b.index:
        if (b.loc[i, 'DD_bef']=='na') and (b.loc[i, 'DD_aft']=='na'):
            b.loc[i, 'Date']=pd.to_datetime(b.loc[i, 'MM']+'/'+'01/'+b.loc[i, 'YYYY'])        
        elif (b.loc[i, 'DD_bef']=='na') and (b.loc[i, 'DD_aft']!='na'):
            b.loc[i,'Date']=pd.to_datetime(b.loc[i,'MM']+'/'+b.loc[i,'DD_aft']+'/'+b.loc[i,'YYYY'])       
        elif (b.loc[i, 'DD_aft']=='na') and (b.loc[i, 'DD_bef']!='na'):
            b.loc[i,'Date']=pd.to_datetime(b.loc[i,'MM']+'/'+b.loc[i,'DD_bef']+'/'+b.loc[i,'YYYY'])      
        else:
            e=('The value at index %s is not valid' % (i))
            #print(e)

    #Convert numbered dates to hh/mm/yyyy
    a=a.rename(columns={1:'MM', 2:'DD', 3:'YYYY'})
    a['DD']=a['DD'].astype(str)
    a['MM']=a['MM'].astype(str)
    a['YYYY']=a['YYYY'].astype(str)
    a['YYYY'] = a['YYYY'].apply(lambda x: '19' + x if len(x)<=2 else x)

    #If date contains dash it always has 3 entries drop any with 2 entries.
    for i in a.index:
        if '-' in a.loc[i, 'DD'] and a.loc[i, 'MM']=='nan':
            a=a.drop(i)

    #MM=DD and is in format mm/yyyy - add DD=01
    for i in a.index:
        if a.loc[i, 'MM']=='nan': 
            a.loc[i, 'MM']=a.loc[i, 'DD']
            a.loc[i, 'DD']='01'

    #Remove any non-sensical years        
    for i in a.index:
        if (len(a.loc[i, 'YYYY'])!=4):
            a=a.drop(i)

    a['DD']=a['DD'].str.rstrip().str.strip('-').str.strip('/').str.pad(width=2)  
    a['MM']=a['MM'].str.rstrip().str.strip('-').str.strip('/').str.pad(width=2) 

    #Remove any non-sensical months 
    for i in a.index:
        if a.loc[i, 'MM']>'12':
             a=a.drop(i)

    #Remove any non-sensical days 
    for i in a.index:
        if a.loc[i, 'DD']>'31':
             a=a.drop(i)        

    #Create date as mm/dd/yyyy     
    a['Date']=pd.to_datetime(a['MM']+'/'+a['DD']+'/'+a['YYYY'])
    #print(len(a)+len(b)+len(cunique))  

    final = pd.concat([a, b, cunique]).sort_values(by='Date').reset_index()
    finalseries=final['level_0']
    return finalseries
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    480
478    286
479    431
480    279
481    198
482    463
483    381
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64